## Задача 1. 
Провести дисперсионный анализ для определения того, есть ли различия среднего роста среди взрослых футболистов, хоккеистов и штангистов.  
Даны значения роста в трех группах случайно выбранных спортсменов:  
Футболисты: 173, 175, 180, 178, 177, 185, 183, 182.  
Хоккеисты: 177, 179, 180, 188, 177, 172, 171, 184, 180.  
Штангисты: 172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170.  

#### 1. Определим гипотезу  

Основная гипотеза $H_0: \mu_1=\mu_2=\mu_3$  
Альтернативная гипотеза $H_1: \mu_1\neq\mu_2\neq\mu_3$  

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import pandas as pd

In [2]:
football = np.array([173, 175, 180, 178, 177, 185, 183, 182])
hockey = np.array([177, 179, 180, 188, 177, 172, 171, 184, 180])
weightlifters = np.array([172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170])

In [3]:
# Определяем уровень значимости для всех тестов
alpha = 0.05

In [4]:
# Проводим тест на нормальность распределения выборок
print(stats.shapiro(football))
print(stats.shapiro(hockey))
print(stats.shapiro(weightlifters))

ShapiroResult(statistic=0.9775082468986511, pvalue=0.9495404362678528)
ShapiroResult(statistic=0.9579196572303772, pvalue=0.7763139009475708)
ShapiroResult(statistic=0.9386808276176453, pvalue=0.5051165223121643)


$p-value > \alpha \Rightarrow H_0$ не отвергается, распределения всех выборок нормальное. Это означает, что предпосылка о нормальности данных для применения ANOVA выполняется.

In [5]:
# Проводим тест на равенство дисперсий
print(stats.bartlett(football, hockey, weightlifters))

BartlettResult(statistic=0.4640521043406442, pvalue=0.7929254656083131)


$p-value > \alpha \Rightarrow H_0$ не отвергается, дисперсии выборок равны. Это подтверждает, что предпосылка о равенстве дисперсий для применения ANOVA выполняется.

In [6]:
# Проводи проверку равенства размерности выборок
print(len(football)==len(hockey)==len(weightlifters))

False


In [7]:
print(len(football))
print(len(hockey))
print(len(weightlifters))

8
9
11


Данные удовлетворяют предпосылкам нормальности и равенства дисперсий, а также имеют умеренное несоответствие в размерах выборок, будем использовать ANOVA для сравнения средних значений между группами.

In [8]:
stats.f_oneway(football, hockey, weightlifters)

F_onewayResult(statistic=5.500053450812596, pvalue=0.010482206918698693)

Cуществует статистически значимая разница между группами (p-value < 0.05).

#### Проводим Post hoc тест Тьюки

In [9]:
df = pd.DataFrame({"score": [173, 175, 180, 178, 177, 185, 183, 182, 
                             177, 179, 180, 188, 177, 172, 171, 184, 180,
                             172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170],
                   "group": np.repeat(["football", "hockey", "weightlifters"], repeats = [8, 9, 11])})
df

,score,group
0,173,football
1,175,football
2,180,football
3,178,football
4,177,football
5,185,football
6,183,football
7,182,football
8,177,hockey
9,179,hockey


In [10]:
tukey = pairwise_tukeyhsd(df["score"], df["group"], alpha = 0.05)
print(tukey)

     Multiple Comparison of Means - Tukey HSD, FWER=0.05      
 group1      group2    meandiff p-adj   lower    upper  reject
--------------------------------------------------------------
football        hockey  -0.4583  0.979  -6.2732  5.3566  False
football weightlifters  -6.3977 0.0219 -11.9583 -0.8372   True
  hockey weightlifters  -5.9394 0.0284 -11.3181 -0.5607   True
--------------------------------------------------------------


#### 5. Выводы
Между группой football и hockey нет различий в среднем росте, тогда как между группами football и weightlifters, а так же между группами hockey и weightlifters есть различия в среднем росте.

Доверительный интервал для разницы средних между группой football и группой weightlifters указывает на то, что он левее нуля, что означает статистически значимое различие между группами, и среднее значение для группы weightlifters меньше, чем для группы football. Следовательно, можно утверждать, что тяжелоатлеты имеют статистически значимо более низкие средние значения, чем футболисты. Данное утверждение справедливо и для групп hockey, weightlifters, таким образом, тяжелоатлеты имеют статистически значимо более низкие средние значения, чем хоккеисты.